# BERT's playground 
Hello there! Welcome on BERT's playground. You may play with BERT here and see what he can do but always make sure he feels respected and admired.

## Setups


In [1]:
import pickle
import tensorflow as tf
import masking
import BERT
import numpy as np

from tensorflow import keras
from Vectorisation import Vectorisation
from Config import Config
from MaskedTextGenerator import MaskedTextGenerator

with open("./ml4science_data.pkl", "rb") as fp:
    data_dict = pickle.load(fp)

config = Config()
vec = Vectorisation(config=config)

In [2]:
# Prepare data for masked language model
encoded = vec.encode_dict(data_dict)
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(encoded, config.TOKEN_DICT, seed=32)
print(x_masked_encoded.shape, y_masked_encoded.shape, sample_weights.shape)

mlm_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded, sample_weights))
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

print(mlm_ds)

(254, 128) (254, 128) (254, 128)
<_BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.float64, name=None))>


In [3]:
sample_tokens = x_masked_encoded[0:1]
print(y_masked_encoded[0])

[26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
  3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
  9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [4]:
print(x_masked_encoded[0])

[26  2 26 21 21 26  9 25 21 23  2 21 26 21 21 26 21  1  3  8  3  1  3  3
  3  3  3  3  7  8  1  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  1  2  8  3  1 10  9 10  9  9 15  1 11  9  1  1 11  9 14 13 14 10  1
  9  9 13 14 13 14 11  9 10 14 13 10 11  1 14  9  1 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [5]:
generator_callback = MaskedTextGenerator(sample_tokens, config.TOKEN_DICT['[MASK]'])

bert_masked_model = BERT.create_masked_language_bert_model(config)
bert_masked_model.summary()


Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 word_embedding (Embedding)  (None, 128, 64)              1728      ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 128, 64)              0         ['word_embedding[0][0]']      
 Lambda)                                                                                          
                                                                                                  
 encoder_0/multiheadattenti  (None, 128, 64)              16640     ['tf.__operat

In [6]:
# A callback in Keras is a function that is called at certain points during the training process. -> Here is called after each epoch during the training
# Here we use to see the "performance" at each epoch while predicting on a "test set" aka the sample_tokens
bert_masked_model.fit(mlm_ds, epochs=100, callbacks=[generator_callback]) 
bert_masked_model.save("bert_models/bert_mlm.keras")

Epoch 1/100
1/1 [==============================] - 0s 151ms/steposs: 3.17

masked nb: 	 1                2                3                4                5                
predictions: 	[21 26 14]       [26 21 10]       [26 14 21]       [26 14 21]       [22 26 21]       
probabilities: 	[0.19 0.14 0.13] [0.21 0.17 0.11] [0.17 0.14 0.12] [0.14 0.11 0.11] [0.22 0.15 0.12] 

16/16 [==============================] - 2s 26ms/step - loss: 3.1173
Epoch 2/100
1/1 [==============================] - 0s 17ms/steploss: 2.74

masked nb: 	 1                2                3                4                5                
predictions: 	[21 26 22]       [21 26 10]       [21 26 22]       [21 26 22]       [21 22 26]       
probabilities: 	[0.3  0.16 0.07] [0.27 0.2  0.08] [0.2  0.16 0.1 ] [0.17 0.13 0.09] [0.18 0.16 0.14] 

16/16 [==============================] - 0s 16ms/step - loss: 2.7299
Epoch 3/100
1/1 [==============================] - 0s 17ms/steploss: 2.64

masked nb: 	 1                2  

In [7]:
predictions = bert_masked_model.predict(x_masked_encoded[0:1])

predictions_max = np.argmax(predictions, axis=2)
print("Predictions:\n",predictions_max)
print("Original:\n", y_masked_encoded[0:1])
print("Masked:\n", x_masked_encoded[0:1])

1/1 [==============================] - 0s 20ms/step
Predictions:
 [[26 21 26 21 21 26 21 25 21 23 21 21 26 21 21 26 21 23  3  8  3  3  3  3
   3  3  3  3  4  8  4  4  8  3 16 20 21 26 21 23  3  3  8  3  8  3  8  8
   8  8  8  8  3  8 10  9 10  9  9 15 14 11  9  9 13 11 14 14 13 14 10 11
   9  9 13 14 13 14 11  9 10 14 13 11 11  9 14  9 14 15 20 15 15 17 20 20
  20  7  7  9  9  9 20 20 20 20 20  8  8  8  8 23 23 13 13 13 15  3 11 11
  10  8  8 15  3  3 14  9]]
Original:
 [[26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
   3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
   4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
   9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]]
Masked:
 [[26  2 26 21 21 26  9 25 21 23  2 21 26 21 21 26 21  1  3  8  3  1  3  3
   3  3  3  3  7  8  1  2  8  3 16 20 21 26 

In [8]:
print(predictions_max == y_masked_encoded[0:1])
# print accuracy
print("Accuracy brut: ", np.sum(predictions_max == y_masked_encoded[0:1]) / (100*len(y_masked_encoded[0:1])))
# print accuracy without padding
print("Accuracy without taking in acount padding: ", np.sum((predictions_max == y_masked_encoded[0:1]) * (y_masked_encoded[0:1] != 0)) / np.sum(y_masked_encoded[0:1] != 0))
print("Accuracy on masked tokens: ", np.sum((predictions_max == y_masked_encoded[0:1]) * (x_masked_encoded[0:1] == config.TOKEN_DICT['[MASK]'])) / np.sum(x_masked_encoded[0:1] == config.TOKEN_DICT['[MASK]']))

[[ True False  True  True  True  True False  True  True  True False  True
   True  True  True  True  True False  True  True  True  True  True  True
   True  True  True  True False  True  True False  True  True  True  True
   True  True  True  True False  True  True False  True  True False  True
  False  True False  True  True  True  True  True  True  True  True  True
  False  True  True False  True  True False  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True False
   True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False]]
Accuracy brut:  0.8
Accuracy without taking in acount padding:  0.8421052631578947
Accuracy on masked tokens:  0.7272727272727273


In [9]:
# This how we can load a Keras model
"""# Load OUR pretrained bert model
mlm_model = keras.models.load_model(
    "bert_mlm_imdb.keras", custom_objects={"MaskedLanguageModel": MaskedLanguageModel}
)"""

# Kinda failed attempt to create a end to end model (we don't really need it) -> But it's a good example of how to create a model with a custom loss function and reshape the output
"""def get_end_to_end(model):
    inputs = keras.Input(shape=(None,))
    outputs = model(inputs)
    reshaped_outputs = keras.layers.Lambda(lambda x: keras.backend.argmax(x, axis=-1))(outputs)
    end_to_end_model = keras.Model(inputs, reshaped_outputs, name="end_to_end_model")
    optimizer = keras.optimizers.Adam(learning_rate=config.bert.LR)
    end_to_end_model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return end_to_end_model

end_to_end_classification_model = get_end_to_end(bert_masked_model)

# Build dataset for end to end model input (will be used at the end)
test_raw_classifier_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded)).batch(config.BATCH_SIZE)

end_to_end_classification_model.evaluate(test_raw_classifier_ds)"""

'def get_end_to_end(model):\n    inputs = keras.Input(shape=(None,))\n    outputs = model(inputs)\n    reshaped_outputs = keras.layers.Lambda(lambda x: keras.backend.argmax(x, axis=-1))(outputs)\n    end_to_end_model = keras.Model(inputs, reshaped_outputs, name="end_to_end_model")\n    optimizer = keras.optimizers.Adam(learning_rate=config.bert.LR)\n    end_to_end_model.compile(\n        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]\n    )\n    return end_to_end_model\n\nend_to_end_classification_model = get_end_to_end(bert_masked_model)\n\n# Build dataset for end to end model input (will be used at the end)\ntest_raw_classifier_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded)).batch(config.BATCH_SIZE)\n\nend_to_end_classification_model.evaluate(test_raw_classifier_ds)'